This Python script retrieves information about packages installed via both conda and pip and exports detailed metadata from PyPI into a CSV file. The metadata includes basic details like the package description and author, as well as additional information such as version, license, long description, project URLs, keywords, supported Python versions, and classifiers (development status, operating system, programming language).

The script also applies various filters to ensure each row in the CSV meets certain criteria. Specifically, it removes rows if:

The package has no version.

The project URLs are empty.

The description is empty.

The first 4 characters of the package name are just whitespace.

Additionally, the script sanitizes all text fields by removing newline characters to avoid multi-line cells in Excel. It also encloses every field in quotes (csv.QUOTE_ALL) to prevent commas or other special characters from splitting fields into multiple columns.

# How It Works
Fetching Package Lists:
The script uses conda list --json to get conda-installed packages and pip list --format=json to get pip-installed packages. These lists are merged to avoid duplicates.

Querying PyPI:
For each unique package, the script queries the PyPI JSON API (https://pypi.org/pypi/<package_name>/json). It extracts several metadata fields:

Basic Information: Summary, author, and license.

Version Information: The current version.

Extended Descriptions: Long description for detailed info.

Requirements & Extras: Lists of dependencies and extras.

Project URLs & Keywords: Additional resources and searchable keywords.

Supported Python Versions: Compatibility details.

Classifiers: Extracts information on development status, operating system, and programming language.

CSV Output:
All the extracted metadata is saved as rows in a CSV file with appropriate column headers.

In [8]:
!conda --version

conda 24.7.1


In [9]:
import subprocess
import json
import requests
import csv
import time

In [25]:
import subprocess
import json
import requests
import csv
import time

def get_conda_packages():
    """Get a set of package names installed via conda."""
    try:
        result = subprocess.run(["conda", "list", "--json"], capture_output=True, text=True, check=True)
        packages = json.loads(result.stdout)
        return {pkg.get("name") for pkg in packages if pkg.get("name")}
    except FileNotFoundError:
        print("Conda executable not found. Make sure conda is installed and added to PATH.")
        return set()
    except subprocess.CalledProcessError as e:
        print("Error fetching conda package list:", e)
        return set()

def get_pip_packages():
    """Get a set of package names installed via pip."""
    try:
        result = subprocess.run(["pip", "list", "--format=json"], capture_output=True, text=True, check=True)
        packages = json.loads(result.stdout)
        return {pkg.get("name") for pkg in packages if pkg.get("name")}
    except subprocess.CalledProcessError as e:
        print("Error fetching pip package list:", e)
        return set()

def sanitize_text(text):
    """
    Remove newline and carriage return characters to keep content in one cell.
    You can also remove or replace other special characters if needed.
    """
    if text:
        return text.replace("\n", " ").replace("\r", " ")
    return text

def get_pypi_metadata(package_name):
    """
    Query PyPI for package metadata and extract fields. 
    Also remove any newlines from all string fields to avoid multi-line CSV issues.
    """
    url = f"https://pypi.org/pypi/{package_name}/json"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            info = data.get("info", {})
            classifiers = info.get("classifiers", [])
            
            # Extract classifier-related details.
            dev_status = ", ".join(
                c.split("::")[-1].strip() 
                for c in classifiers 
                if c.startswith("Development Status")
            )
            operating_system = ", ".join(
                c.split("::")[-1].strip() 
                for c in classifiers 
                if c.startswith("Operating System")
            )
            prog_lang = ", ".join(
                c.split("::")[-1].strip() 
                for c in classifiers 
                if c.startswith("Programming Language")
            )
            
            # Process requires.
            requires = info.get("requires_dist")
            requires = "; ".join(requires) if requires else ""
            
            # Additional metadata
            version = info.get("version", "")
            license_info = info.get("license", "")
            project_urls = info.get("project_urls", {})
            project_urls_str = "; ".join(
                f"{k}: {v}" for k, v in project_urls.items()
            ) if project_urls else ""
            requires_python = info.get("requires_python", "")
            description = info.get("summary", "")
            
            # Build the metadata dictionary
            metadata = {
                "package": package_name,
                "version": version,
                "description": description,
                "author": info.get("author", ""),
                "license": license_info,
                "requires": requires,
                "project_urls": project_urls_str,
                "requires_python": requires_python,
                "development_status": dev_status,
                "operating_system": operating_system,
                "programming_language": prog_lang
            }
            
            # Sanitize all string fields to remove newlines.
            for key, value in metadata.items():
                if isinstance(value, str):
                    metadata[key] = sanitize_text(value)
            
            return metadata
        else:
            # Return empty fields if the package is not found on PyPI
            return {
                "package": package_name,
                "version": "",
                "description": "",
                "author": "",
                "license": "",
                "requires": "",
                "project_urls": "",
                "requires_python": "",
                "development_status": "",
                "operating_system": "",
                "programming_language": ""
            }
    except Exception as e:
        print(f"Error retrieving metadata for {package_name}: {e}")
        return {
            "package": package_name,
            "version": "",
            "description": "",
            "author": "",
            "license": "",
            "requires": "",
            "project_urls": "",
            "requires_python": "",
            "development_status": "",
            "operating_system": "",
            "programming_language": ""
        }

def main():
    # 1. Get package names from conda and pip.
    conda_packages = get_conda_packages()
    pip_packages = get_pip_packages()
    
    # 2. Combine the sets of packages.
    all_packages = conda_packages.union(pip_packages)
    print(f"Found {len(conda_packages)} conda packages and {len(pip_packages)} pip packages.")
    print(f"Total unique packages to process: {len(all_packages)}")
    
    # 3. Fetch metadata from PyPI
    results = []
    print("Fetching metadata from PyPI...")
    for pkg_name in sorted(all_packages):
        metadata = get_pypi_metadata(pkg_name)
        results.append(metadata)
        # Delay to be courteous to the PyPI API
        time.sleep(0.1)
    
    # 4. Filter out rows that do not meet your criteria:
    #    - 'version', 'project_urls', 'description' must not be empty
    #    - first 4 characters of 'package' must not be just whitespace
    filtered_results = []
    for data in results:
        pkg_name = data.get("package", "")
        # Condition checks
        if (data.get("version") and
            data.get("project_urls") and
            data.get("description") and
            pkg_name[:4].strip() != ""):
            filtered_results.append(data)
    
    print(f"After filtering, {len(filtered_results)} packages remain with valid fields.")
    
    # 5. Define CSV columns
    fieldnames = [
        "package",
        "version",
        "description",
        "author",
        "license",
        "requires",
        "project_urls",
        "requires_python",
        "development_status",
        "operating_system",
        "programming_language"
    ]
    
    # 6. Write results to a CSV file with QUOTE_ALL
    csv_filename = "combined_packages_metadata.csv"
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(
            csvfile, 
            fieldnames=fieldnames, 
            delimiter=',', 
            quoting=csv.QUOTE_ALL
        )
        writer.writeheader()
        for data in filtered_results:
            writer.writerow(data)
    
    print(f"CSV file '{csv_filename}' created with metadata for {len(filtered_results)} packages.")
    print("Tip: In Excel, use 'Data → From Text/CSV' and select comma as the delimiter,")
    print("     and double quotes as the text qualifier to avoid splitting rows.")

if __name__ == "__main__":
    main()


Conda executable not found. Make sure conda is installed and added to PATH.
Found 0 conda packages and 246 pip packages.
Total unique packages to process: 246
Fetching metadata from PyPI...
After filtering, 243 packages remain with valid fields.
CSV file 'combined_packages_metadata.csv' created with metadata for 243 packages.
Tip: In Excel, use 'Data → From Text/CSV' and select comma as the delimiter,
     and double quotes as the text qualifier to avoid splitting rows.
